<a class="anchor" id="0"></a>
# Predictive Analysis of Cryptocurrencies

<a class="anchor" id="0.1"></a>

## Table of Contents

1. [Import libraries and set parameters](#1)
1. [Download data](#2)
1. [EDA](#3)
   - [3.1 Market Cap](#3.1)
   - [3.2 Cryptocurrency data](#3.2)
   - [3.3 Cryptocurrency features data](#3.3)
   - [3.4 Stationarity check](#3.4)
   - [3.5 Identification of seasonality](#3.5)
   - [3.6 EDA with Pandas Profiling Report](#3.6)   
1. [FE](#4)
   - [4.1 FE with TSFRESH](#4.1)
   - [4.2 FE from technical features (Finance knowledge and Data Science)](#4.2)
   - [4.3 Analysis of anomalies](#4.3)
       - [4.3.1 Analysis of anomalies for "Close"](#4.3.1)
       - [4.3.2 Analysis of anomalies for the first data difference "Close_diff"](#4.3.2)
   - [4.4 Analysis of the impact of COVID-19 on the cryptocurrency rate](#4.4)
   - [4.5 Get target, training, validation and test datasets for ML models](#4.5)
1. [Model training and forecasting](#5)
    - [5.1 Facebook Prophet](#5.1)
    - [5.2 ARIMA](#5.2)
        - [5.2.1 How to find the order of differencing (d) in ARIMA model](#5.2.1)
        - [5.2.2 How to find the order of the AR term (p)](#5.2.2)
        - [5.2.3 How to find the order of the MA term (q)](#5.2.3)
        - [5.2.4 How to build the ARIMA Model with manually defined parameters](#5.2.4)
        - [5.2.5 How to build the ARIMA automatically](#5.2.5)
    - [5.3 Other ML models (Multi-factors models)](#5.3)
        - [5.3.1 Set parameters for many models](#5.3.1)
        - [5.3.2 Models training and forecasting](#5.3.2)
    - [5.4 Choosing the main optimal model and forecasting](#5.4)
    - [5.5 Feature importance study](#5.5)    

## 1. Import libraries and set parameters <a class="anchor" id="1"></a>

In [ ]:
# Import libraries
import random
import os
import numpy as np 
import pandas as pd 
import requests
import pandas_datareader as web

# Date
import datetime as dt
from datetime import date, timedelta, datetime

# EDA
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import pandas_profiling as pp

# FE
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.inspection import permutation_importance
import eli5
from eli5.sklearn import PermutationImportance
import shap

# Time Series - EDA and Modelling
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Modeling and preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from prophet import Prophet
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# What EDA & FE techniques use?
is_EDA_with_Pandas_Profiling = True # or False - Get Pandas Profiling Report or no?
is_EDA_with_COVID19_data = True # or False - Make EDA with COVID-19 data or no?
is_anomalies = True # or False - Take into account anomalies or no?

In [ ]:
# What type of model to use?
is_Prophet = True   # or False - Facebook Prophet
is_ARIMA = True     # or False - ARIMA and AutoARIMA
is_other_ML = True  # or False - multi-factors models: trees, neural networks, etc.

In [ ]:
# Automatic building ARIMA for Time Series
if is_ARIMA:
    !pip install pmdarima
    import pmdarima as pm

In [ ]:
# Set random state
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

random_state = 42
fix_all_seeds(random_state)

**TASK:** It is proposed to experiment with forecasting_days

In [ ]:
# Set main parameters
cryptocurrency = 'BTC'
target = 'Close'
forecasting_days = 10  # forecasting_days > 1

**TASK :** It is proposed to experiment with date_start and date_end

In [ ]:
# Set time interval of data for given cryptocurrency - the period of coronavirus in 2020-2021
date_start = dt.datetime(2020, 4, 1)
# date_end = dt.datetime.now()
date_end = dt.datetime(2021, 12, 31)
print(f"Time interval: from {date_start} to {date_end}")

## 2. Download data <a class="anchor" id="2"></a>


In [ ]:
# Download information about cryptocurrencies
df_about = pd.read_csv("../input/forecasting-top-cryptocurrencies/about_top_cryptocurrencies_1B_information.csv", sep=";")
display(df_about)

In [ ]:
def get_part_number(x):
    # Get 1 - st, 2 - nd, 3 - rd, 4.. - th in the first, second, third, ...
    if x==1:
        return 'st'
    elif x==2:
        return 'nd'
    elif x==3:
        return 'rd'
    else: return 'th'

In [ ]:
def get_rank_cryptocurrency(cryptocurrency):
    # Get rank by Market Cap for code of the cryptocurrency
    # Download the dataset from https://coinmarketcap.com/currencies/bitcoin/
    
    place = df_about.index[df_about['code'] == cryptocurrency].tolist()[0]
    print(f"{df_about.loc[place, 'name']} was {place+1}{get_part_number(place+1)}",
          "among the world's cryptocurrencies by market capitalization (2022-04-11)")
    
# Get rank by Market Cap of the cryptocurrency
get_rank_cryptocurrency(cryptocurrency)

In [ ]:
def get_data(cryptocurrency, date_start, date_end=None):
    # Get data for given cryptocurrency in USD from Yahoo.finance and https://coinmarketcap.com/
    # date_end = None means that the date_end is the current day
    
    if date_end is None:
        date_end = dt.datetime.now()
    df = web.DataReader(f'{cryptocurrency}-USD', 'yahoo', date_start, date_end)
    
    return df

# Download data of the cryptocurrency via API
df = get_data(cryptocurrency, date_start, date_end)
df

In [ ]:
# Correlation coefficients
df.corr()

In [ ]:
# Correlation coefficients
df.corr()['Close']

In [ ]:
df = df.drop(columns = ["Adj Close"])
df

## 3. EDA <a class="anchor" id="3"></a>

[Back to Table of Contents](#0.1)

### 3.1. Market Cap <a class="anchor" id="3.1"></a>


In [ ]:
# Market Cap
crypto = pd.Series(df_about.market_cap.head(20).tolist(), index=df_about.name.head(20).tolist(), name="Market capitalization")
crypto.plot.pie(figsize=(10, 10))

### 3.2. Cryptocurrency data <a class="anchor" id="3.2"></a>

In [ ]:
df['Close'].plot(grid=True, figsize=(12,8))

### 3.3. Cryptocurrency features data <a class="anchor" id="3.3"></a>


In [ ]:
display(df)

In [ ]:
def c_chart(data,label):

    candlestick = go.Figure(data = [go.Candlestick(x=data.index,
                                                   open = data['Open'], 
                                                   high = data['High'], 
                                                   low = data['Low'], 
                                                   close = data['Close'])])
    candlestick.update_xaxes(title_text = 'Time',
                             rangeslider_visible = True)

    candlestick.update_layout(
    title = {
            'text': '{:} Candelstick Chart'.format(label),
            "y":0.8,
            "x":0.5,
            'xanchor': 'center',
            'yanchor': 'top'})

    candlestick.update_yaxes(title_text = 'Price in USD', ticksuffix = '$')
    return candlestick

In [ ]:
%matplotlib inline
btc_candle=c_chart(df, label="BTC Price")
btc_candle.show()

### 3.4. Stationarity check <a class="anchor" id="3.4"></a>

Thanks to [Time Series: Interpreting ACF and PACF](https://www.kaggle.com/code/iamleonie/time-series-interpreting-acf-and-pacf)

ACF and PACF assume stationarity of the underlying time series.
Staionarity can be checked by performing an **Augmented Dickey-Fuller (ADF) test**:

> - p-value > 0.05: Fail to reject the null hypothesis (H0), the data has a unit root and is non-stationary.
> - p-value <= 0.05: Reject the null hypothesis (H0), the data does not have a unit root and is stationary.
>
> [...] We can see that our [ADF] statistic value [...] is less than the value [...] at 1%.
This suggests that we can reject the null hypothesis with a significance level of less than 1% (i.e. a low probability that the result is a statistical fluke).
Rejecting the null hypothesis means that the process has no unit root, and in turn that the time series is stationary or does not have time-dependent structure. - [Machine Learning Mastery: How to Check if Time Series Data is Stationary with Python](https://machinelearningmastery.com/time-series-data-stationary-python/)

If the time series is stationary, continue to the next steps.
**If the time series is not stationary, try differencing the time series** and check its stationarity again.


In [ ]:
def check_stationarity(series):
    # Thanks to https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

In [ ]:
# Stationarity check
check_stationarity(df['Close'])

In [ ]:
# Stationarity check of the first difference of time series
check_stationarity(df['Close'].diff().dropna())

In [ ]:
# Stationarity check of the second difference of time series
check_stationarity(df['Close'].diff().diff().dropna())

Therefore, it is necessary to model the first difference of the series:

In [ ]:
df['Close_diff'] = df['Close'].diff()
df = df.dropna()
df

### 3.5. Identification of seasonality <a class="anchor" id="3.5"></a>


In [ ]:
# Get seasonality of the time series
decomp = seasonal_decompose(df.Close)
fig = decomp.plot()
fig.set_size_inches((12, 10))
fig.tight_layout()
plt.show()

In [ ]:
# Get seasonality of last months (Dec 2021) of the time series
decomposition = seasonal_decompose(df.tail(30).Close)
fig = decomposition.plot()
fig.set_size_inches((12, 10))
fig.tight_layout()
plt.grid(True)
plt.show()

There is a weekly seasonality, but its contribution is very small, so it can be neglected.

### 3.6. EDA with Pandas Profiling Report <a class="anchor" id="3.6"></a>


In [ ]:
%%time
if is_EDA_with_Pandas_Profiling:
    profile = df.profile_report(title='Pandas Profiling Report for dataset')
    profile.to_file(output_file="profile.html")
    display(profile)

## 4. FE <a class="anchor" id="4"></a>

[Back to Table of Contents](#0.1)

### 4.1. FE with TSFRESH<a class="anchor" id="4.1"></a>

[Back to Table of Contents](#0.1)

In [ ]:
def get_tsfresh_features(data):
    # Get statistic features using library TSFRESH 
    
    data = data.reset_index(drop=False).reset_index(drop=False)
    
    # Extract features
    extracted_features = extract_features(data, column_id="Date", column_sort="Date")
    
    # Drop features with NaN
    extracted_features_clean = extracted_features.dropna(axis=1, how='all').reset_index(drop=True)
    
    # Drop features with constants
    cols_std_zero  = []
    for col in extracted_features_clean.columns:
        if extracted_features_clean[col].std()==0:
            cols_std_zero.append(col)
    extracted_features_clean = extracted_features_clean.drop(columns = cols_std_zero)

    extracted_features_clean['Date'] = data['Date']   # For the merging
    
    return extracted_features_clean

In [ ]:
%%time
# FE with TSFRESH
extracted_features_clean = get_tsfresh_features(df[['Close']])
extracted_features_clean

In [ ]:
extracted_features_clean.describe()

In [ ]:
# Extracted features by TSFRESH with cleaning
extracted_features_clean.columns.tolist()

In [ ]:
# Get all features
df = pd.merge(df, extracted_features_clean, how='left', on='Date')
df

In [ ]:
df.shape

### 4.2. FE from technical features (Finance knowledge and Data Science)<a class="anchor" id="4.2"></a>

[Back to Table of Contents](#0.1)

The description of these features and more complex options are described in the scientific paper **Mokin V.B., etc."Information Technology for the Cryptocurrency Rate Forecasting on the Basics of Complex Feature Engineering". [Visnyk VPI](https://visnyk.vntu.edu.ua/index.php/visnyk). No 2 (2022).**

In [ ]:
def get_add_features(df_feat):
    # FE for data as row of DataFrame
    
    # Two new features from the competition tutorial
    df_feat['Upper_Shadow'] = df_feat['High'] - np.maximum(df_feat['Close'], df_feat['Open'])
    df_feat['Lower_Shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    
    # Thanks to https://www.kaggle.com/code1110/gresearch-simple-lgb-starter
    df_feat['lower_shadow'] = np.minimum(df_feat['Close'], df_feat['Open']) - df_feat['Low']
    df_feat['high2low'] = (df_feat['High'] / df_feat['Low']).replace([np.inf, -np.inf, np.nan], 0.)
    
    return df_feat

**TASK :** It is proposed to experiment with FE : add new features and modify existing ones

In [ ]:
# FE - add features
df = get_add_features(df)
df

### 4.3. Analysis of anomalies<a class="anchor" id="4.3"></a>

#### 4.3.1. Analysis of anomalies for "Close"<a class="anchor" id="4.3.1"></a>

In [ ]:
# Drawing plot with Plotly
if is_anomalies:
    fig = px.line(df, x="Date", y="Close", 
                  title=f"Investigation of dates of anomalous changes in the cryptocurrency rate", 
                  log_y=False,template='gridon',width=800, height=600)
    fig.show()

In [ ]:
# Synthesis dataframe with anomalous dates for Facebook Prophet
if is_anomalies:
    anomalous_dates = ['2022-01-08', '2022-01-27', '2022-04-13', '2022-07-20',
                       '2022-09-06', '2022-09-29', '2022-11-08', '2022-12-17']
    holidays_df = pd.DataFrame(columns = ['ds', 'lower_window', 'upper_window', 'prior_scale'])
    holidays_df['ds'] = anomalous_dates
    holidays_df['holiday'] = 'anomalous_dates'
    holidays_df['lower_window'] = 0
    holidays_df['upper_window'] = 0
    holidays_df['prior_scale'] = 10
    display(holidays_df)

In [ ]:
def plot_with_anomalies(df, cols_y_list, cols_y_list_name, dates_x, anomalous_dates, log_y=False):
    # Draws a plot with title - the features cols_y_list (y) and dates_x (x) from the dataframe df
    # and with vertical lines in the dates from the list anomalous_dates
    # with the length between the minimum and maximum of feature cols_y_list[0]
    # with log_y = False or True
    # cols_y_list - dictionary of the names of cols from cols_y_list (keys - name of feature, value - it's name for the plot legend), 
    # name of cols_y_list[0] is the title of the all plot
    
    fig = px.line(df, x=dates_x, y=cols_y_list[0], title=cols_y_list_name[cols_y_list[0]], log_y=log_y, template='gridon',width=800, height=600)
    y_max = df[cols_y_list[0]].max()
    for i in range(len(cols_y_list)-1):
        fig.add_trace(go.Scatter(x=df[dates_x], y=df[cols_y_list[i+1]], mode='lines', name=cols_y_list_name[cols_y_list[i+1]]))
        max_i = df[cols_y_list[i+1]].max()
        y_max = max_i if max_i > y_max else y_max
    
    y_min = min(df[cols_y_list[0]].min(),0)
    for i in range(len(anomalous_dates)):
        anomal_date = anomalous_dates[i]
        #print(anomal_date, y_min, y_max)
        fig.add_shape(dict(type="line", x0=anomal_date, y0=y_min, x1=anomal_date, y1=y_max, line=dict(color="red", width=1)))
    fig.show()

In [ ]:
# Draw plot
if is_anomalies:
    plot_with_anomalies(df, ["Close"], 
                        {"Close" : f"Anomalous dates for {cryptocurrency}"}, 
                        'Date', anomalous_dates, False)

If we simulate the first data difference "Close_diff", then other anomalies will be added, when not just an unexpected change, but when it was very large.

#### 4.3.2. Analysis of anomalies for the first data difference "Close_diff"<a class="anchor" id="4.3.2"></a>

In [ ]:
# Drawing plot with Plotly
if is_anomalies:
    fig = px.line(df, x="Date", y="Close_diff", 
                  title=f"Investigation of dates of anomalous changes in the first difference of the cryptocurrency rate", 
                  log_y=False,template='gridon',width=800, height=600)
    fig.show()

In [ ]:
# Add new anomalous dates
if is_anomalies:
    anomalous_dates_diff = anomalous_dates.copy()
    anomalous_dates_diff.append('2021-02-08')
    anomalous_dates_diff.append('2021-05-12')
    anomalous_dates_diff.append('2021-09-07')
    print(anomalous_dates_diff)

In [ ]:
# Synthesis dataframe with anomalous dates for Facebook Prophet
if is_anomalies:
    holidays_df_diff = pd.DataFrame(columns = ['ds', 'lower_window', 'upper_window', 'prior_scale'])
    holidays_df_diff['ds'] = anomalous_dates_diff
    holidays_df_diff['holiday'] = 'anomalous_dates_for_difference'
    holidays_df_diff['lower_window'] = 0
    holidays_df_diff['upper_window'] = 0
    holidays_df_diff['prior_scale'] = 10
    display(holidays_df_diff)

In [ ]:
# Draw plot
if is_anomalies:
    plot_with_anomalies(df, ["Close_diff"], 
                        {"Close_diff" : f"Anomalous dates for the first difference of the {cryptocurrency}"}, 
                        'Date', anomalous_dates_diff, False)

In [ ]:
# Synthesis a new feature in df for anomalous_dates_diff
if is_anomalies:
    df['Close_diff_anomalous'] = df['Date'].isin(anomalous_dates_diff).astype('int')    
    display(df)
    
    # Number of anomalous dates
    print(f"Number of anomalous dates - {df['Close_diff_anomalous'].sum()}")

### 4.4. Analysis of the impact of COVID-19 on the cryptocurrency rate <a class="anchor" id="4.4"></a>

In [ ]:
# Set COVID parameters
if is_EDA_with_COVID19_data:
    covid_feature = 'New_Deaths'  # or "New_Cases"
    country_covid_feature = f"USA_{covid_feature}"
    print('country_covid_feature =', country_covid_feature)

In [ ]:
def get_covid_data(date_start, covid_feature, country='USA'):

    # Source: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
    
    if covid_feature=='New_Cases':
        file = "time_series_covid19_confirmed_global.csv"
        name_feature = 'Cases'
    elif covid_feature=="New_Deaths":
        file = "time_series_covid19_deaths_global.csv"
        name_feature = 'Deaths'
    
    myfile = requests.get(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{file}')
    open('data', 'wb').write(myfile.content)
    global_df = pd.read_csv('data')
    
    if country=='USA':
        code = 'US'
    else: code = country
    
    try:
        global_df = global_df[global_df['Country/Region']==code]
    except:
        print('Non-existent country code given')
        return None

    def convert_date_str(df):
        try:
            df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
        except:
            print('_convert_date_str failed with %y, try %Y')
            df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]

    convert_date_str(global_df)
    
    global_df2 = global_df.melt(
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], value_vars=global_df.columns[4:], var_name='Date', value_name=name_feature)

    df_covid = global_df2[['Date', name_feature]]
    df_covid[name_feature] = df_covid[name_feature].astype('int').diff()
    df_covid = df_covid.fillna(0)

    df_covid['ds'] = pd.to_datetime(df_covid['Date'])
    df_covid = df_covid[df_covid['ds'] > date_start][['ds', name_feature]].reset_index(drop=True)
    df_covid.columns = ['Date', country_covid_feature]

    return df_covid

if is_EDA_with_COVID19_data:
    df_covid = get_covid_data(date_start, covid_feature)

In [ ]:
def df_covid_data_imputing(df_covid):
    # Imputing COVID data for USA

    def pd_imputing(df, date1, date2, col):
        x1 = float(df[df['Date']==date1][col].head(1))
        x2 = float(df[df['Date']==date2][col].head(1))
        return (x1+x2)/2

    def df_add(df, date_middle, date1, date2, col=country_covid_feature):
        # Add imputed COVID data for USA
        df = df.append({'Date': datetime.strptime(date_middle, '%Y-%m-%d'), col : pd_imputing(df, date1, date2, col=col)}, ignore_index=True)
        return df

    # Only for USA - the imputing missing data
    date_anomal = ['2021-10-08', '2021-10-11', '2021-10-12', '2021-10-25']
    df_covid = df_add(df_covid, '2021-10-08', '2021-10-07', '2021-10-09')
    df_covid = df_add(df_covid, '2021-10-11', '2021-10-10', '2021-10-13')
    df_covid = df_add(df_covid, '2021-10-12', '2021-10-11', '2021-10-14')
    df_covid = df_add(df_covid, '2021-10-25', '2021-10-24', '2021-10-26')
    df_covid = df_covid.sort_values(by=['Date']).reset_index(drop=True)
    
    return df_covid

if is_EDA_with_COVID19_data:
    df_covid = df_covid_data_imputing(df_covid)
    display(df_covid)

In [ ]:
if is_EDA_with_COVID19_data:
    data = pd.merge(df[['Date', 'Close']], df_covid, on = 'Date')
    data.index = data['Date']
    display(data)

In [ ]:
def draw_crypto_and_covid(data):
    # Displays COVID data in USA and cryptocurrency data on one plot
    
    def df_minmax_scaler(df):
        # Data Scalling
        index_df = df.pop('Date')
        scaler = MinMaxScaler().fit(df)
        df = pd.DataFrame(scaler.transform(df), columns = df.columns, index = index_df)
        return df

    data = df_minmax_scaler(data.copy())

    # Data smoothing and visualization
    cols_scaled = ['Close_Smoothed_Scaled', country_covid_feature + "_Smoothed_Scaled"]
    data.columns = cols_scaled
    for col in cols_scaled:
        data[col] = data[col].rolling(7).mean()
    data[cols_scaled].plot(lw=4, grid=True, figsize=(12,10))

if is_EDA_with_COVID19_data:
    draw_crypto_and_covid(data)

In [ ]:
# Saving the dataset
if is_EDA_with_COVID19_data:
    df.to_csv(f'data_of_{cryptocurrency}.csv', index=False)

### 4.5. Get target, training, validation and test datasets for ML models<a class="anchor" id="4.5"></a>

In [ ]:
# Illustration of number transformations in the columns:
# "Close" -> "Close_diff" -> "Target" -> "Close_diff_pred" -> "Close_pred"
# Get target and the result of the forecasting
forecasting_days_example = 3
df_example = pd.DataFrame({'Close':[1, 2, 4, 8, 15, 25], 'Day': [0, 1, 2, 3, 4, 5]})
df_example['Close_diff'] = df_example['Close'].diff()
df_example['target'] = df_example['Close_diff'].shift(-forecasting_days_example)
df_example['target_pred'] = df_example['target'].copy()   # Ideal forecasting result
print(f'Simulation of the result of ideal forecasting the "target_pred" for {forecasting_days_example} days')
display(df_example[['Day', 'Close', 'Close_diff', 'target', 'target_pred']])

# Get inverse target
print('\nSimulation of the recovering predicted values "Close_pred" from the "target_pred"')
df_example['Close_diff_pred_shifted'] = df_example['target_pred'].shift(forecasting_days_example)

# Let's create an intermediate feature to make it easier to explain the transformation
temp_column_name = f'Close_diff_pred_shifted_with_Close'  # Intermediate feature for transformations 
df_example[temp_column_name] = df_example['Close_diff_pred_shifted'].copy()
df_example.loc[forecasting_days_example, temp_column_name] = df_example.loc[forecasting_days_example,'Close']
df_example['Close_pred'] = np.concatenate((df_example['Close'].tolist()[:forecasting_days_example], 
                                           np.cumsum(df_example[temp_column_name].values[forecasting_days_example:], dtype=float)))
df_example['Close_pred'] = df_example['Close_pred'].astype('int')
display(df_example[['Day', 'Close', 'Close_diff', 'target', 'target_pred', 'Close_diff_pred_shifted', temp_column_name, 'Close_pred']])

1. Recovery is possible if you know exactly at least one value "Close".
2. If the last N values for the time series will be in the target_test, then during the comparing with the output of other multi-features ML models needs remember that in them the target is shifted back and taken with a difference.
3. Target for Time series model is "Close". 
4. Target is for multi-factors models is "target_pred" = shifted "Close_diff".

In [ ]:
def cut_data(df, y, num_start, num_end):
    # Cutting dataframe df and array or list for [num_start, num_end-1]        
    df2 = df[num_start:(num_end+1)]
    y2 = y[num_start:(num_end+1)] if y is not None else None
    return df2, y2

In [ ]:
def get_target_mf(df, forecasting_days, col='Close'):
    # Get target as difference of the df[col] 
    # Returns target which is shifted for forecasting_days days in the dataframe df
    # "Close" -> "Close_diff" -> "Target" 
    col_diff = f"{col}_diff"
    df[col_diff] = df['Close'].diff()
    df['target'] = df[col_diff].shift(-forecasting_days)
    df = df.drop(columns=[col_diff]).dropna()
    
    return df

In [ ]:
def get_train_valid_test_ts(df, forecasting_days, target='Close'):
    # Get training, validation and test datasets with target for Time Series models
    
    # Data prepairing
    df = df.dropna(how="any").reset_index(drop=True)
    df = df[['Date', 'Close']]
    df.columns = ['ds', 'y']        
    y = None

    # Data smoothing
#     df.index = df.ds
#     df = df.drop(columns=['ds'])
#     df['y'] = df['y'].rolling(7).mean()
#     df = df.dropna().reset_index(drop=False)
    
    N = len(df)
    train, _ = cut_data(df, y, 0, N-2*forecasting_days-1)
    valid, _ = cut_data(df, y, N-2*forecasting_days, N-forecasting_days-1)
    test, _ = cut_data(df, y, N-forecasting_days, N)
    
    # Train+valid - for optimal model training
    train_valid = pd.concat([train, valid])

    print(f'Origin dataset has {len(df)} rows and {len(df.columns)} features')
    print(f'Get training dataset with {len(train)} rows')
    print(f'Get validation dataset with {len(valid)} rows')
    print(f'Get test dataset with {len(test)} rows')
    
    return train, valid, test, train_valid

In [ ]:
def get_train_valid_test_mf(df, forecasting_days, target='target'):
    # Get training, validation and test datasets with target for multi-features ML models
    
    df = df.drop(columns = ['Date']).dropna(how="any").reset_index(drop=True)
    
    # Save and drop target        
    y = df.pop(target)

    # Get starting points for the recovering "Close" from "Close_diff_shigted"
    N = len(df)
    #print(f"Total - {N}, Valid start index = {N-forecasting_days-1}, Test start index = {N-1}")
    start_points = {'valid_start_point' : df.loc[N-forecasting_days-1, 'Close'],
                    'test_start_point' : df.loc[N-1, 'Close']}

    # Standartization data
    scaler = StandardScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
    
    
    train, ytrain = cut_data(df.copy(), y, 0, N-2*forecasting_days-1)
    valid, yvalid = cut_data(df.copy(), y, N-2*forecasting_days, N-forecasting_days-1)
    test, ytest = cut_data(df.copy(), y, N-forecasting_days, N)


    # Train+valid - for optimal model training
    train_valid = pd.concat([train, valid])
    y_train_valid = pd.concat([ytrain, yvalid])

    print(f'Origin dataset has {len(df)} rows and {len(df.columns)} features')
    print(f'Get training dataset with {len(train)} rows')
    print(f'Get validation dataset with {len(valid)} rows')
    print(f'Get test dataset with {len(test)} rows')
    
    return train, ytrain, valid, yvalid, test, ytest, train_valid, y_train_valid, start_points

## 5. Model training and forecasting <a class="anchor" id="5"></a>

This section provides examples of identifying the following models (but the list goes on):
* Facebook Prophet 
* ARIMA (and AutoARIMA)
* Linear Regression
* KNeighbors Regressor
* Support Vector Machines
* Linear SVR
* Random Forest Regressor
* Bagging Regressor
* XGB Regressor
* MLP Regressor

FB Prophet and ARIMA models have a slightly different data format, while other Machine Learning (ML) models have the same data, so it is easy to increase their number.

Classic model XGBoost have a special format, but this notebook  uses its simplified version, which work in a data format similar to the models of the Sklearn library.

Models based on neural networks (based on PyTorch or Keras) and ensembles of all these models are more effective, but this will be done later in other notebooks.

In [ ]:
def calc_metrics(type_score, list_true, list_pred):
    # Calculation score with type=type_score for list_true and list_pred 
    if type_score=='r2_score':
        score = r2_score(list_true, list_pred)
    elif type_score=='rmse':
        score = mean_squared_error(list_true, list_pred, squared=False)
    elif type_score=='mape':
        score = mean_absolute_percentage_error(list_true, list_pred)
    return score

In [ ]:
def result_add_metrics(result, n, y_true, y_pred):
    # Calculation and addition metrics into dataframe result[n,:]
    
    result.loc[n,'r2_score'] = calc_metrics('r2_score', y_true, y_pred)
    result.loc[n,'rmse'] = calc_metrics('rmse', y_true, y_pred)      # in coins
    result.loc[n,'mape'] = 100*calc_metrics('mape', y_true, y_pred)  # in %
    
    return result

In [ ]:
# Results of all models
result = pd.DataFrame(columns = ['name_model', 'type_data', 'r2_score', 'rmse', 'mape', 'params', 'ypred'])

### 5.1.Prophet <a class="anchor" id="5.1"></a>

In [ ]:
# Modeling 2022 year only
if is_Prophet:
    df2 = df[df.Date.dt.year == 2022]
    display(df2)

In [ ]:
# Get datasets
if is_Prophet:
    train_ts, valid_ts, test_ts, train_valid_ts = get_train_valid_test_ts(df2.copy(), forecasting_days, target='Close')
    
    if not is_anomalies:
        holidays_df = None

In [ ]:
def prophet_modeling(result, 
                     cryptocurrency, 
                     train, 
                     test, 
                     holidays_df, 
                     period_days,
                     fourier_order_seasonality,
                     forecasting_period,
                     name_model,
                     type_data):
    # Performs FB Prophet model training for given train dataset, holidays_df and seasonality_mode
    # Performs forecasting with period by this model, visualization and error estimation
    # df - dataframe with real data in the forecasting_period
    # can be such combinations of parameters: train=train, test=valid or train=train_valid, test=test
    # Save results into dataframe result
    
    # Build Prophet model with parameters and structure 
    model = Prophet(daily_seasonality=False, 
                    weekly_seasonality=False, 
                    yearly_seasonality=False, 
                    changepoint_range=1, 
                    changepoint_prior_scale = 0.5, 
                    holidays=holidays_df, 
                    seasonality_mode = 'multiplicative'
                   )
    model.add_seasonality(name='seasonality', period=period_days, 
                          fourier_order=fourier_order_seasonality, 
                          mode = 'multiplicative', prior_scale = 0.5)
    # Training model for df
    model.fit(train)
    
    # Make a forecast
    future = model.make_future_dataframe(periods = forecasting_period)
    forecast = model.predict(future)
    
    # Draw plot of the values with forecasting data
    figure = model.plot(forecast, xlabel = 'Date', ylabel = f"{name_model} for {cryptocurrency}")
    
    # Draw plot with the components (trend and seasonalities) of the forecasts
    figure_component = model.plot_components(forecast)
    
    # Ouput the prediction for the next time on forecasted_days
    #forecast[['yhat_lower', 'yhat', 'yhat_upper']] = forecast[['yhat_lower', 'yhat', 'yhat_upper']].round(1)
    #forecast[['ds', 'yhat_lower', 'yhat', 'yhat_upper']].tail(forecasting_period)
    
    # Forecasting data by the model
    ypred = forecast['yhat'][-forecasting_period:]
    #print(ypred)
    # Save results
    n = len(result)
    result.loc[n,'name_model'] = f"Prophet_{name_model}"
    result.loc[n,'type_data'] = type_data
    result.at[n,'params'] = [period_days]+[fourier_order_seasonality]
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, test['y'], y_pred)
    
    return result, ypred

**TASK :** It is proposed to experiment with models parameters

In [ ]:
%%time
# Models tuning
if is_Prophet:
    for period_days in [4, 5, 7, 14]:
        for fourier_order_seasonality in [3, 12]:
            result, _ = prophet_modeling(result, 
                                         cryptocurrency, 
                                         train_ts, 
                                         valid_ts, 
                                         holidays_df, 
                                         period_days,
                                         fourier_order_seasonality,
                                         forecasting_days,
                                         f'{period_days}_days_{fourier_order_seasonality}_order',
                                         'valid')

### 5.2. ARIMA <a class="anchor" id="5.2"></a>

This information from the good notebook [ARIMA Model for Time Series Forecasting](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting)

ARIMA stands for Autoregressive Integrated Moving Average Model. It belongs to a class of models that explains a given time series based on its own past values -i.e.- its own lags and the lagged forecast errors. The equation can be used to forecast future values. Any ‘non-seasonal’ time series that exhibits patterns and is not a random white noise can be modeled with ARIMA models.
So, ARIMA, short for AutoRegressive Integrated Moving Average, is a forecasting algorithm based on the idea that the information in the past values of the time series can alone be used to predict the future values.
ARIMA Models are specified by three order parameters: (p, d, q),

where,

- p is the order of the AR term

- d is the number of differencing required to make the time series stationary

- q is the order of the MA term


- AR(p) Autoregression – a regression model that utilizes the dependent relationship between a current observation and observations over a previous period. An auto regressive (AR(p)) component refers to the use of past values in the regression equation for the time series.

- I(d) Integration – uses differencing of observations (subtracting an observation from observation at the previous time step) in order to make the time series stationary. Differencing involves the subtraction of the current values of a series with its previous values d number of times.

- MA(q) Moving Average – a model that uses the dependency between an observation and a residual error from a moving average model applied to lagged observations. A moving average component depicts the error of the model as a combination of previous error terms. The order q represents the number of terms to be included in the model.

In [ ]:
# Get datasets
if is_ARIMA:
    train_ts, valid_ts, test_ts, train_valid_ts = get_train_valid_test_ts(df2.copy(), forecasting_days, target='Close')

#### **5.2.1 How to find the order of differencing (d) in ARIMA model**  <a class="anchor" id="5.2.1"></a>

This information from the good notebook [ARIMA Model for Time Series Forecasting](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting)

- As stated earlier, the purpose of differencing is to make the time series stationary. But we should be careful to not over-difference the series. An over differenced series may still be stationary, which in turn will affect the model parameters.


- So we should determine the right order of differencing. The right order of differencing is the minimum differencing required to get a near-stationary series which roams around a defined mean and the ACF plot reaches to zero fairly quick.


- If the autocorrelations are positive for many number of lags (10 or more), then the series needs further differencing. On the other hand, if the lag 1 autocorrelation itself is too negative, then the series is probably over-differenced.


- If we can’t really decide between two orders of differencing, then we go with the order that gives the least standard deviation in the differenced series.


- Now, we will explain these concepts with the help of an example as follows:


- First, I will check if the series is stationary using the **Augmented Dickey Fuller test (ADF Test)**, from the statsmodels package. The reason being is that we need differencing only if the series is non-stationary. Else, no differencing is needed, that is, d=0.


- The null hypothesis (Ho) of the ADF test is that the time series is non-stationary. So, if the p-value of the test is less than the significance level (0.05) then we reject the null hypothesis and infer that the time series is indeed stationary.


- So, in our case, if P Value > 0.05 we go ahead with finding the order of differencing.

- **A similar analysis has already been made in the paragraph "3.4. Stationarity check" above**

In [ ]:
def acf_pacf_draw(df, lag_num=40, acf=True, pacf=True, title="", ylim=1):
    # Draw plots named title with ACF and PACF for dataframe df
    
    num_plots = 1+int(acf)+int(pacf)
    fig, ax = plt.subplots(1,num_plots,figsize=(12,6))
    # 'Original Series'
    ax[0].plot(df.values.squeeze())
    
    if acf:
        # ACF drawing
        plot_acf(df.values.squeeze(), lags=lag_num, ax=ax[1])
        ax[1].set(ylim=(-ylim, ylim))
        
        if pacf:
            # PACF drawing
            plot_pacf(df.values.squeeze(), lags=lag_num, ax=ax[2])
            ax[2].set(ylim=(-ylim, ylim))
        
    elif pacf:
        # PACF drawing
        plot_pacf(df.values.squeeze(), lags=lag_num, ax=ax[1])
        ax[1].set(ylim=(-ylim, ylim))

    fig.suptitle(title)
    plt.show()

In [ ]:
if is_ARIMA:
    # ACF and PACF
    lag_num = 100
    acf_pacf_draw(train_ts['y'], lag_num, True, True, 'Original Series')
    acf_pacf_draw(train_ts['y'].diff().dropna(), lag_num, True, True, '1st Order Differencing')
    acf_pacf_draw(train_ts['y'].diff().diff().dropna(), lag_num, True, True, '2nd Order Differencing')

- For the above data, we can see that the time series reaches stationarity with first orders of differencing. Although, it value may be higher, as can be seen from the larger values of the lag.

In [ ]:
d = 1

#### **5.2.2. How to find the order of the AR term (p)** <a class="anchor" id="5.2.2"></a>

[Table of Contents](#0.1)

This information from the good notebook [ARIMA Model for Time Series Forecasting](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting)

- The next step is to identify if the model needs any AR terms. We will find out the required number of AR terms by inspecting the **Partial Autocorrelation (PACF) plot**.


- **Partial autocorrelation** can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. This way, we will know if that lag is needed in the AR term or not.


- Partial autocorrelation of lag (k) of a series is the coefficient of that lag in the autoregression equation of Y.


$$Yt = \alpha0 + \alpha1 Y{t-1} + \alpha2 Y{t-2} + \alpha3 Y{t-3}$$


- That is, suppose, if Y_t is the current series and Y_t-1 is the lag 1 of Y, then the partial autocorrelation of lag 3 (Y_t-3) is the coefficient $\alpha_3$ of Y_t-3 in the above equation.


- Now, we should find the number of AR terms. Any autocorrelation in a stationarized series can be rectified by adding enough AR terms. So, we initially take the order of AR term to be equal to as many lags that crosses the significance limit in the PACF plot.



In [ ]:
# PACF drawing
if is_ARIMA:
    acf_pacf_draw(train_ts['y'].diff().dropna(), 30, False, True, '1st Order Differencing', 1)

- We can see that the PACF lag 0 is quite significant since it is well above the significance line. So, we will fix the value of p as 1. Although, it value may be higher, as can be seen from the larger values of the lag.

In [ ]:
p = 0

#### **5.2.3. How to find the order of the MA term (q)** <a class="anchor" id="5.2.3"></a>

This information from the good notebook [ARIMA Model for Time Series Forecasting](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting)

- Just like how we looked at the PACF plot for the number of AR terms, we will look at the ACF plot for the number of MA terms. An MA term is technically, the error of the lagged forecast.


- The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series.


- Let’s see the autocorrelation plot of the differenced series.

In [ ]:
# ACF drawing
if is_ARIMA:
    acf_pacf_draw(train_ts['y'].diff().dropna(), 30, True, False, '1st Order Differencing', 1)

- We can see that couple of lags are well above the significance line. So, we will fix q as 0. If there is any doubt, we will go with the simpler model that sufficiently explains the Y.

In [ ]:
q = 0

#### **5.2.4. How to build the ARIMA Model with manually defined parameters** <a class="anchor" id="5.2.4"></a>


Now, we have determined the values of p, d and q. We have everything needed to fit the ARIMA model. We will use the ARIMA() implementation in the statsmodels package.

In [ ]:
def arima_fit(df, col, order=(1,1,1)):
    # ARIMA model fitting for series df[col]
    
    model = sm.tsa.arima.ARIMA(df[col].values.squeeze(), order=order)
    model = model.fit()
    return model

In [ ]:
if is_ARIMA:
    # ARIMA Model tuning
    model = arima_fit(train_ts, 'y', order=(p,d,q))
    print(model.summary())

- The model summary provides lot of information. The table in the middle is the coefficients table where the values under ‘coef’ are the weights of the respective terms.

- The model AIC has slightly reduced, which is good. The p-values  in ‘P>|z|’ column is highly significant (<< 0.05).

In [ ]:
if is_ARIMA:
    # ARIMA model diagnostics
    fig = model.plot_diagnostics(figsize=(12,10))
    plt.show()

#### **Interpretation of plots in plot diagnostics**

This information from the good notebook [ARIMA Model for Time Series Forecasting](https://www.kaggle.com/code/prashant111/arima-model-for-time-series-forecasting)

**Standardized residual**: The residual errors seem to fluctuate around a mean of zero and have a uniform variance.


**Histogram**: The density plot suggest normal distribution with mean slighlty shifted towards right.


**Theoretical Quantiles**: Mostly the dots fall perfectly in line with the red line. Any significant deviations would imply the distribution is skewed.


**Correlogram**: The Correlogram, (or ACF plot) shows the residual errors are not autocorrelated. The ACF plot would imply that there is some pattern in the residual errors which are not explained in the model. So we will need to look for more X’s (predictors) to the model.


Overall, the model seems to be a good fit. Let’s plot the residuals to ensure there are no patterns (that is, look for constant mean and variance).

In [ ]:
def get_residual_errors(model):
    # Calculation and drawing the plot residual errors for ARIMA model
    residuals = pd.DataFrame(model.resid)
    fig, ax = plt.subplots(1,2, figsize=(12,6))
    residuals.plot(title="Residuals", ax=ax[0])
    residuals.plot(kind='kde', title='Density', ax=ax[1])
    plt.show()

In [ ]:
if is_ARIMA:
    # Plot residual errors
    get_residual_errors(model)

- The residual errors seem fine with near zero mean and uniform variance.

In [ ]:
def arima_forecasting(result, model, params, name_model, df, type_data):
    # Data df (validation or test) forecasting on the num days by the model 
    # with params and save metrics to result 
    
    ypred = model.forecast(steps=len(df))
    
    n = len(result)
    result.loc[n,'name_model'] = name_model
    result.loc[n,'type_data'] = type_data
    result.at[n,'params'] = params
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, df['y'], y_pred)
    
    return result

In [ ]:
if is_ARIMA:
    # Valid forecasting and save result
    result = arima_forecasting(result, model, [p]+[d]+[q], 'ARIMA_manual', valid_ts, 'valid')

#### **5.2.5. How to build the ARIMA automatically** <a class="anchor" id="5.2.5"></a>

You can try to use additional features in the parameter X of pm.auto_arima to improve forecasting, but this will be done in other notebooks later.

In [ ]:
%%time
if is_ARIMA:
    # Automatic tuning of the ARIMA model 
    model_auto = pm.auto_arima(train_ts['y'].values, 
                               start_p=4,        # start p
                               start_q=4,        # start q
                               test='adf',       # use adftest to find optimal 'd'
                               max_p=5, max_q=5, # maximum p and q
                               m=1,              # frequency of series (1 - No Seasonality)
                               d=None,           # let model determine 'd'
                               seasonal=False,   # No Seasonality
                               start_P=0,        
                               D=0, 
                               start_Q=0,
                               trace=True,
                               error_action='ignore',  
                               suppress_warnings=False, 
                               stepwise=True     # use the stepwise algorithm outlined in Hyndman and Khandakar (2008) 
                                                 # to identify the optimal model parameters. 
                                                 # The stepwise algorithm can be significantly faster than fitting all 
                                                 # hyper-parameter combinations and is less likely to over-fit the model
                              )

    print(model_auto.summary())

In [ ]:
if is_ARIMA:
    # Get orders of the best model from AutoARIMA
    arima_orders_best = list(model_auto.get_params().get('order'))
    print(f"Optimal parameters are {arima_orders_best}")
    model_auto = arima_fit(train_ts, 'y', order=(arima_orders_best[0],arima_orders_best[1],arima_orders_best[2]))

In [ ]:
if is_ARIMA:
    # Best model from AutoARIMA
    fig = model_auto.plot_diagnostics(figsize=(12,10))
    plt.show()

- The residual errors seem fine with near zero mean and uniform variance.

In [ ]:
if is_ARIMA:
    # Valid forecasting and save result
    result = arima_forecasting(result, model_auto, arima_orders_best, 'ARIMA_auto', valid_ts, 'valid')

### 5.3. Other ML models (Multi-factors models) <a class="anchor" id="5.3"></a>

This section provides examples of identifying the following models (but the list goes on):
* Linear Regression
* KNeighbors Regressor
* Support Vector Machines
* Linear SVR
* Random Forest Regressor
* Bagging Regressor
* XGB Regressor
* MLP Regressor

Classic model XGBoost have a special format, but this notebook  uses its simplified version, which work in a data format similar to the models of the Sklearn library.

Models based on neural networks (based on PyTorch or Keras) and ensembles of all these models are more effective, but this will be done later in other notebooks.

**This section - from the notebook [Crypto - BTC : 7 prediction models](https://www.kaggle.com/code/vbmokin/crypto-btc-7-prediction-models)**

In [ ]:
# Get datasets
if is_other_ML:
    df2 = get_target_mf(df2, forecasting_days, col='Close')
    train_mf, ytrain_mf, valid_mf, yvalid_mf, test_mf, ytest_mf, train_valid_mf, y_train_valid_mf, starting_point = \
                                    get_train_valid_test_mf(df2.copy(), forecasting_days, target='target')

#### **5.3.1. Set parameters for many models** <a class="anchor" id="5.3.1"></a>

**TASK:** Try adding more models or changing the settings of these models.

In [ ]:
if is_other_ML:
    # Set parameters of models
    models = pd.DataFrame(columns = ['name', 'model', 'param_grid'])

    # Linear Regression
    n = len(models)
    models.loc[n, 'name'] = 'Linear Regression'
    models.at[n, 'model'] = LinearRegression()
    models.at[n, 'param_grid'] = {'fit_intercept' : [True, False]}


    # KNeighbors Regressor
    n = len(models)
    models.loc[n, 'name'] = 'KNeighbors Regressor'
    models.at[n, 'model'] = KNeighborsRegressor()
    models.at[n, 'param_grid'] = {'n_neighbors': [3, 5, 10, 20, 30],
                                  'leaf_size': [10, 20, 30]
                                 }

    # Support Vector Machines
    n = len(models)
    models.loc[n, 'name'] = 'Support Vector Machines'
    models.at[n, 'model'] = SVR()
    models.at[n, 'param_grid'] = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                  'C': np.linspace(1, 15, 15),
                                  'tol': [1e-3, 1e-4]
                                 }

    # Linear SVC
    n = len(models)
    models.loc[n, 'name'] = 'Linear SVR'
    models.at[n, 'model'] = LinearSVR()
    models.at[n, 'param_grid'] = {'C': np.linspace(1, 15, 15)}


    # Random Forest Classifier
    n = len(models)
    models.loc[n, 'name'] = 'Random Forest Regressor'
    models.at[n, 'model'] = RandomForestRegressor()
    models.at[n, 'param_grid'] = {'n_estimators': [40, 50, 60, 80], 
                                  'min_samples_split': [30, 40, 50, 60], 
                                  'min_samples_leaf': [10, 12, 15, 20, 50],
                                  'max_features': ['auto'], 
                                  'max_depth': [3, 4, 5, 6]                   
                                 }

    # Bagging Classifier
    n = len(models)
    models.loc[n, 'name'] = 'Bagging Regressor'
    models.at[n, 'model'] = BaggingRegressor()
    models.at[n, 'param_grid'] = {'max_features': np.linspace(0.05, 0.8, 1),
                                  'n_estimators': [3, 4, 5, 6],
                                  'warm_start' : [False]
                                 }

    # XGB Classifier
    n = len(models)
    models.loc[n, 'name'] = 'XGB Regressor'
    models.at[n, 'model'] = xgb.XGBRegressor()
    models.at[n, 'param_grid'] = {'n_estimators': [50, 70, 90], 
                                  'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                  'max_depth': [3, 4, 5]
                                 }

    # MLP Classifier
    n = len(models)
    models.loc[n, 'name'] = 'MLP Regressor'
    models.at[n, 'model'] = MLPRegressor()
    models.at[n, 'param_grid'] = {'hidden_layer_sizes': [i for i in range(2,5)],
                                  'solver': ['lbfgs', 'sgd'],
                                  'learning_rate': ['adaptive'],
                                  'learning_rate_init': [0.001, 0.01],
                                  'max_iter': [1000]
                                 }
models

#### **5.3.2. Models training and forecasting** <a class="anchor" id="5.3.2"></a>

In [ ]:
def model_prediction(result, models, train_features, valid_features, train_labels, valid_labels):    
    # Models training and data prediction for all models from DataFrame models
    # Saving results for validation dataset into dataframe result
    
    def calc_add_score(res, n, type_score, list_true, list_pred, feature_end):
        # Calculation score with type=type_score for list_true and list_pred 
        # Adding score into res.loc[n,...]
        res.loc[i, type_score + feature_end] = calc_metrics(type_score, list_true, list_pred)
        return res
    
    # Results
    model_all = []

    for i in range(len(models)):
        # Training
        print(f"Tuning model '{models.loc[i, 'name']}'")
        model = GridSearchCV(models.at[i, 'model'], models.at[i, 'param_grid'])
        model.fit(train_features, train_labels)
        model_all.append(model)
        print(f"Best parameters: {model.best_params_}\n")
        
        # Prediction
        ypred = model.predict(valid_features)
        
        # Scoring and saving results into the main dataframe result
        n = len(result)
        result.loc[n,'name_model'] = f"{models.loc[i, 'name']}"
        result.loc[n,'type_data'] = "valid"
        result.at[n,'params'] = model.best_params_
        result.at[n,'ypred'] = ypred
        #result = result_add_metrics(result, n, valid_labels, valid_pred)
        
    return result, model_all

In [ ]:
%%time
if is_other_ML:
    # Models tuning and the forecasting
    result, model_all = model_prediction(result, models, train_mf, valid_mf, ytrain_mf, yvalid_mf)

### 5.4. Choosing the main optimal model and forecasting <a class="anchor" id="5.4"></a>

In [ ]:
def recovery_prediction(y, starting_point):
    # Recovering prediction of multi-factors model for shifted col_diff to col in the dataframe df
    # y has type np.array
    # starting_point is dictionary with start values for the recovering data
    # Returns y (np.array) with recovering data
    
    return np.insert(y, 0, starting_point).cumsum()[1:]

In [ ]:
def result_recover_and_metrics(result, df_ts, type_data, start_points):
    # Recovering prediction: from shifted_Close_diff to Close
    # Calculation metrics for recovering ypred forecasting for all models in result
    # ypred real is from df_ts['y']
    # start points value for the recovering is from dictionary start_points
    # type_data = 'valid' or 'test'

    for i in range(len(result)):
        if (result.loc[i, 'type_data']==type_data) and (result.loc[i, 'mape'] is np.nan):
            ypred = result.loc[i, 'ypred']

            # Recovering ypred for multi-factors models
            if not (str(result.loc[i, 'type_model']) in ['Prophet', 'ARIMA']):
                # Multi-factors model
                # Get start points value for the recovering
                start_point_value = start_points['valid_start_point'] if type_data=='valid' else start_points['test_start_point']
                # Recovering prediction
                ypred = recovery_prediction(ypred, start_point_value)            

            # Calculation metrics
            result = result_add_metrics(result, i, df_ts['y'], ypred)
    
    return result

In [ ]:
# Dispay and save all results for validation dataset
if len(result) > 0:
    
    # Get type of each model
    result['type_model'] = result['name_model'].str.split('_').str[0]

    # Calculation metrics for recovering prediction ypred for validation dataset by all models 
    result = result_recover_and_metrics(result, valid_ts, 'valid', starting_point)
    display(result[['name_model', 'type_data', 'r2_score', 'rmse', 'mape']].sort_values(by=['type_data', 'mape', 'rmse'], ascending=True))
    
    # Save results
    num_models = len(result[result['type_data']=='valid']['name_model'].unique().tolist())
    print(f"Number of models built - {num_models}")
    result.to_csv(f'result_of_{num_models}_models_for_forecasting_days_{forecasting_days}.csv')
else: 
    print('There are no tuned models!')

In [ ]:
def get_model_opt(name_model, params):
    # Model tuning for the name_model
    
    print(name_model)
    if name_model=='Linear Regression':
        model = LinearRegression(**params)
        
    elif name_model=='KNeighbors Regressor':
        model = KNeighborsRegressor(**params)
        
    elif name_model=='Support Vector Machines':
        model = SVR(**params)
        
    elif name_model=='Linear SVR':
        model = LinearSVR(**params)
        
    elif name_model=='Random Forest Regressor':
        model = RandomForestRegressor(**params)
        
    elif name_model=='Bagging Regressor':
        model = BaggingRegressor(**params)
    
    elif name_model=='MLP Regressor':
        model = MLPRegressor(**params)
        
    elif name_model=='XGB Regressor':
        model = xgb.XGBRegressor(**params)
        
    else: model = None
        
    return model

In [ ]:
def get_params_optimal_model(result, main_metrics):
    # Get parameters of the optimal model from dataframe result by main_metrics

    # Set the data type to float (just in case)
    result[main_metrics] = result[main_metrics].astype('float')
    
    # Choose the optimal model
    opt_result = result[result['type_data']=='valid'].reset_index(drop=True)
    if main_metrics=='r2_score':
        opt_model = opt_result.nlargest(1, main_metrics)
    else:
        # 'mape' or 'rmse'
        opt_model = opt_result.nsmallest(1, main_metrics)
    display(opt_model[['name_model', 'r2_score', 'rmse', 'mape', 'params']])

    # Get parameters of the optimal model
    opt_name_model = opt_model['name_model'].tolist()[0]
    opt_type_model = opt_model['type_model'].tolist()[0]
    opt_params_model = opt_model['params'].tolist()[0]
    print(f'Optimal model by metrics "{main_metrics}" is "{opt_name_model}" with type "{opt_type_model}" parameters {opt_params_model}')
    
    return opt_name_model, opt_type_model, opt_params_model

In [ ]:
def model_training_forecasting(result, df, y, test, ytest,  
                               name_model, type_model, params, type_test='1'):
    # Model training for df and y
    # Forecasting ypred
    # type_model = 'Prophet' or "ARIMA" or 'Other ML'
    # type_test = '1' (with find optimal parameters by GridSearchCV) 
    # type_test = '2' (with optimal parameters - without GridSearchCV)
    # return params and metrics in the dataframe result
    
    if type_model=='Prophet':    
        season_days_optimal = params[0]
        fourier_order_seasonality_optimal = params[1]
        model_opt = None
        _, ypred = prophet_modeling(result, 
                                    cryptocurrency, 
                                    df, 
                                    test, 
                                    holidays_df, 
                                    season_days_optimal,
                                    fourier_order_seasonality_optimal,
                                    forecasting_days,
                                    f'{type_model}_optimal',
                                    'test')        
    elif type_model=='ARIMA':
        season_days_optimal = params[0]
        fourier_order_seasonality_optimal = params[1]        
        model_opt = None
        
        # Training ARIMA optimal model for training+valid dataset
        df['y'] = y
        model_opt = arima_fit(df, 'y', order=(params[0],params[1],params[2]))        

        # Model diagnostics
        fig = model_opt.plot_diagnostics(figsize=(12,10))
        plt.show()

        # Plot residual errors
        get_residual_errors(model_opt)

        # Test forecasting and save result
        ypred = model_opt.forecast(steps=len(test))        

    else:
        # Other ML model
        # Training ML optimal model for training+valid dataset
        print(f"Tuning model '{name_model}'")
        models_opt_number = models[models['name']==name_model].index.tolist()[0]
        #print(f"Model - {models.at[models_opt_number,'model']} with parameters {params}")
        if type_test=='1':
            model_opt = GridSearchCV(models.at[models_opt_number,'model'], models.at[models_opt_number,'param_grid'])
        else:
            # type_test=='2'
            model_opt = get_model_opt(models.at[models_opt_number,'name'], params)
        model_opt.fit(df, y)
        
        # Forecasting
        ypred = model_opt.predict(test)

        
    # Scoring and saving results into the dataframe result
    n = len(result)-1
    result.loc[n,'name_model'] = f"{type_model}_optimal"
    result.loc[n,'type_data'] = "test"
    result.loc[n,'type_model'] = type_model
    result.at[n,'params'] = params
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, ytest, ypred)
    
    return result, model_opt, ypred

In [ ]:
def get_optimal_model_and_forecasting(result, main_metrics, start_points):
    # Choosion the optimal model from dataframe result by main_metrics
    # Tuning optimal model for big dataset train+valid 
    # Test forecasting and drawing it
    # Returns the optimal model and it's name

    
    if len(result) > 0:
        # Get parameters of the optimal model from dataframe result by main_metrics
        opt_name_model, opt_type_model, opt_params_model = get_params_optimal_model(result, 
                                                                                    main_metrics)
        # Set datasets for the final tuning and testing by optimal model
        if (opt_type_model=='Prophet') or (opt_type_model=='ARIMA'):
            train_valid = train_valid_ts.copy()
            y_train_valid = train_valid_ts['y'].copy()
            test = test_ts.copy()
            ytest = test_ts['y'].copy()
            
        else:
            # Multi-factors ML models
            train_valid = train_valid_mf.copy()
            y_train_valid = y_train_valid_mf.copy()
            test = test_mf.copy()
            ytest = ytest_mf.copy()
    
        # Optimal model training for train+valid and test forecasting
        result, model_opt, ypred = model_training_forecasting(result, train_valid, y_train_valid,
                                                              test, ytest,
                                                              opt_name_model, opt_type_model, 
                                                              opt_params_model, '1')
        
        # Calculation metrics for recovering prediction ypred for test dataset by the optimal model
        result = result_recover_and_metrics(result, test_ts, 'test', start_points)
        
        # Drawing plot for prediction for the test data 
        if not ((opt_type_model=='Prophet') or (opt_type_model=='ARIMA')):
            # Recovery values "Close"
            ytest_plot = recovery_prediction(ytest.values, start_points['test_start_point'])
            ypred_plot = recovery_prediction(ypred, start_points['test_start_point'])
        else:
            ytest_plot = ytest.copy()
            ypred_plot = ypred.copy()
            
        # Drawing 
        plt.figure(figsize=(12,8))
        x = np.arange(len(ytest_plot))
        plt.scatter(x, ytest_plot, label = "Target test data", color = 'g', s=100)
        plt.scatter(x, ypred_plot, label = f"{opt_name_model} forecasting", color = 'r', s=50)
        plt.title(f'Forecasting of test data using the "{opt_name_model}" model, which is optimal for "{main_metrics}" metrics')
        plt.ylim(0)
        plt.legend(loc='lower right')
        plt.grid(True)
        
        return opt_name_model

In [ ]:
result

In [ ]:
# Get the optimal model by different metrics
if len(result) > 0:
    for valid_metrics in ['r2_score', 'rmse', 'mape']:
        get_optimal_model_and_forecasting(result, valid_metrics, starting_point)    

### 5.5. Feature importance study <a class="anchor" id="5.5"></a>

A feature importance study is performed for the best of the "non Тime Series" models, as no additional features were used in the Time Series models.

In [ ]:
# Training ML optimal model for training+valid dataset
# Get parameters of the optimal model from dataframe result (without Time Series models) by main_metrics
if is_other_ML:
    main_metrics = 'r2_score'
    if (len(result) > 0) and (len(models) > 0):
        result_nonTS = result[(result['type_model']!='Prophet') & (result['type_model']!='ARIMA')].reset_index(drop=True)
        opt_name_model2, opt_type_model2, opt_params_model2 = get_params_optimal_model(result_nonTS, 
                                                                                main_metrics)

        result, model_opt, ypred = model_training_forecasting(result, 
                                                              train_valid_mf, 
                                                              y_train_valid_mf,
                                                              test_mf, 
                                                              ytest_mf,
                                                              opt_name_model2, 
                                                              opt_type_model2, 
                                                              opt_params_model2, 
                                                              '2')

In [ ]:
# All features names
if is_other_ML:
    coeff = pd.DataFrame(train_valid_mf.columns)
    coeff.columns = ['feature']

In [ ]:
def add_fi_coeff(coeff, col, list_new_fi_coeff=None, df_new_fi_coeff=None):
    # Adds new importance of features as feature col
    # from list list_new_fi_coeff or dataframe df_new_fi_coeff
    # to the resulting dataframe coeff with feature names 
    # Missed importance values are replaced by zero
    
    if list_new_fi_coeff is not None:
        df_new_fi_coeff = coeff[['feature']].copy()
        df_new_fi_coeff["score"] = pd.Series(list_new_fi_coeff)
    
    if df_new_fi_coeff is not None:
        # Rename df_new_fi_coeff
        df_new_fi_coeff.columns = ['feature', 'score']   # to the plot drawing
        df_new_fi_coeff[col] = df_new_fi_coeff['score']  # to the merging and saving
        
        # Merging dataframes - coeff of all features with new_fi_coeff
        coeff = coeff.merge(df_new_fi_coeff[['feature', col]], on='feature', how='left').fillna(0)
        
        is_score = True
    else:
        print(f'Data is absent fol {col}')
        is_score = False
        coeff = None
    
    return coeff, df_new_fi_coeff, is_score

In [ ]:
# Feature importance diagram with SHAP
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        print('Feature importance diagram with SHAP:')
        try:
            # Trees
            explainer = shap.TreeExplainer(model_opt)
            shap_values = explainer.shap_values(test_mf)
            shap.summary_plot(shap_values, test_mf, plot_type="bar", feature_names=coeff['feature'].tolist())
            shap.summary_plot(shap_values, test_mf)

            # Save permutation feature importance values
            coeff, _, is_SHAP_successfully = add_fi_coeff(coeff, 'shap_fi_score', shap_values)
        except: 
            try:
                # Other types of models
                explainer = shap.KernelExplainer(model_opt.predict, train_valid_mf)
                shap_values = explainer.shap_values(test_mf)

                # Plot drawing
                shap.summary_plot(shap_values, test_mf, plot_type="bar", feature_names=coeff['feature'].tolist())
                shap.summary_plot(shap_values, test_mf)

                # Get feature importance values from shap_values format
                # Thanks to https://stackoverflow.com/a/69523421/12301574
                shap_values_all = pd.DataFrame(shap_values, columns = test_mf.columns)
                vals = np.abs(shap_values_all.values).mean(0)
                shap_importance = pd.DataFrame(list(zip(test_mf.columns, vals)),
                                                  columns=['feature','score'])            

                # Saving feature importance values
                coeff, _, is_SHAP_successfully = add_fi_coeff(coeff, 'shap_fi_score', None, shap_importance)            

            except: 
                is_SHAP_successfully = False

        if not is_SHAP_successfully:
            print('Feature importance diagram for this optimal model is not supported in SHAP')

In [ ]:
# Force plot - Feature importance diagram with SHAP for the certaion row in test_mf
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        row_number_in_test_mf = 0
        print('Feature importance diagram as the Force plot with SHAP:')
        if is_SHAP_successfully:
            shap.initjs()
            shap.force_plot(explainer.expected_value, shap_values[0,:], 
                            test_mf.loc[test_mf.index.tolist()[row_number_in_test_mf],:],
                            feature_names=coeff['feature'].tolist(),
                            matplotlib=True, show=False)
            plt.savefig('force_plot.png')

In [ ]:
# Creation and drawing the feature importance diagrams
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):

        # Coefficients
        if opt_name_model2=='XGB Regressor':
            print('Feature importance diagram')
            # Coef. of the feature with nonzero importance
            xgb_coeff = pd.DataFrame.from_dict(model_opt.get_booster().get_score(importance_type='weight'), orient='index').reset_index(drop=False)
            coeff, _, is_score = add_fi_coeff(coeff, 'xgb_fi_coeff', None, xgb_coeff)

            # With the library xgboost
            fig =  plt.figure(figsize = (15,15))
            axes = fig.add_subplot(111)
            xgb.plot_importance(model_opt,ax = axes,height = 0.5)
            plt.show()
            plt.close()

        else:
            # With the library sklearn
            try:
                coef_model = model_opt.coef_
                coeff, coeff_new, is_score = add_fi_coeff(coeff, 'lr_fi_score', coef_model)
            except:
                try:
                    coef_model = feature_importances_
                    coeff, coeff_new, is_score = add_fi_coeff(coeff, 'model_fi_score', coef_model)
                except: 
                    print('The importance of the feature could not be obtained')
                    is_score = False

            if is_score:
                # Plot drawing
                coeff_non_zero = coeff_new[coeff_new['score']>0]
                plt.figure(figsize=(12, int(len(coeff_non_zero)*0.4)))
                coeff_non_zero = coeff_non_zero.sort_values(by='score', ascending=True)
                plt.barh(coeff_non_zero["feature"], coeff_non_zero["score"])
                plt.title("Feature importance diagram")
                plt.axvline(x=0, color=".5")
                plt.xlabel("Coefficient values")
                plt.subplots_adjust(left=0.3)

In [ ]:
# Permutation feature importance diagram
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        try:
            perm_importance = permutation_importance(model_opt, test_mf, ytest_mf)

            # Save permutation feature importance values
            coef_model = perm_importance.importances_mean
            coeff, coeff_new, is_score = add_fi_coeff(coeff, 'perm_fi_score', coef_model)

            print('Permutation feature importance diagram:') 
            coeff_non_zero = coeff_new[coeff_new['score'].abs()>1e-4]
            coeff_non_zero = coeff_non_zero.sort_values(by='score', ascending=True)
            plt.figure(figsize=(12, int(len(coeff_non_zero)*0.4)))
            plt.barh(coeff_non_zero["feature"], coeff_non_zero["score"])
            plt.xlabel("Permutation Importance")
            plt.show()
            is_perm_importance = True
        except: print('Permutation feature importance diagram for this optimal model is not supported')

In [ ]:
# Feature importance diagram with ELI5
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        try:
            print('Feature importance diagram with ELI5:')
            perm = PermutationImportance(model_opt).fit(test_mf,ytest_mf)

            # Save permutation feature importance values
            coef_model = perm.feature_importances_  # Feature importances, 
                                                    # computed as mean decrease 
                                                    # of the score when a feature 
                                                    # is permuted (i.e. becomes noise)
            coeff, _, is_score = add_fi_coeff(coeff, 'eli5_perm_fi_score', coef_model)

            # Display permutation feature importance values with ELI5
            display(eli5.show_weights(perm, feature_names = coeff.feature.tolist()))

        except: print('Feature importance diagram for this optimal model is not supported in ELI5')

In [ ]:
# Display and saving features importance values
if is_other_ML:
    if coeff.isna().sum().sum()==0:
        print('Feature importance values:')
        fi_cols = coeff.columns.tolist()[1:]
        if len(fi_cols) > 0:
            coeff = coeff.sort_values(by=fi_cols, ascending=False)
            display(coeff)
        coeff.to_csv(f'feature_importance_for_optimal_model_{opt_name_model2}.csv', index=False)